# 1710. Maximum Units on a Truck

[Leetcode 1710 Maximum Units on a Truck](https://leetcode.com/problems/maximum-units-on-a-truck/description/) is one of the problems on the [Amazon Questions List](https://leetcode.com/problem-list/7p5x763/?sorting=W3sic29ydE9yZGVyIjoiREVTQ0VORElORyIsIm9yZGVyQnkiOiJGUkVRVUVOQ1kifV0%3D&page=1)

## In this notebook
We're going to go over all the logic necessary to solve the problem (hopefully) as fast as possible, as well as (some of) the observations required to get there.

The problem text:

> You are assigned to put some amount of boxes onto one truck.
> 
> You are given a 2D array boxTypes, where boxTypes<sub>i</sub >= [numberOfBoxes<sub>i</sub>, numberOfUnitsPerBox<sub>i</sub>]:
>
> - numberOfBoxes<sub>i</sub> is the number of boxes of type i.
> - numberOfUnitsPerBox<sub>i</sub> is the number of units in each box of the type i.
>
> You are also given an integer truckSize, which is the maximum number of boxes that can be put on the truck. You can choose any boxes to put on the truck as long as the number of boxes does not exceed truckSize.
>
>Return the maximum total number of units that can be put on the truck.
>
> Constraints:
> - 1 <= boxTypes.length <= 1000
> - 1 <= numberOfBoxes<sub>i</sub>, numberOfUnitsPerBox<sub>i</sub> <= 1000
> - 1 <= truckSize <= 10**6

Makes perfect sense that Amazon would care about a question involving boxes.

So where do we start?



# Observation(s) Step

Let's examine those constraints again:
> Constraints:
> - 1 <= boxTypes.length <= 1000
> - 1 <= numberOfBoxes<sub>i</sub>, numberOfUnitsPerBox<sub>i</sub> <= 1000
> - 1 <= truckSize <= 10**6

The box types and the number of boxes are both relatively small, clocking it at less than 1000 each.

Truck size is slightly large, at 100_000 or less, so we'll call that medium.

Let's also make the observation that any individual box can contain any number of units and doesn't take up additional space. It's still just a box, and we're only concerned about the number of boxes. Intuitively, a 12"x12"x12" box full of books isn't any larger than a 12"x12"x12" box of chocolate, even if the latter is far tastier.

## Question: Do we just take as many of the biggest box as we can every time?

Example 1:
> Input: boxTypes = [[1,3],[2,2],[3,1]], truckSize = 4
> 
> Output: 8
>
> Explanation: There are:
> - 1 box of the first type that contains 3 units.
> - 2 boxes of the second type that contain 2 units each.
> - 3 boxes of the third type that contain 1 unit each.
> You can take all the boxes of the first and second types, and one box of the third type.
> The total number of units will be = (1 * 3) + (2 * 2) + (1 * 1) = 8.

That all makes sense and they also showed their work by showing the numberOfUnitsPerBox going in descending order.

So working through Example 2:

> Input: boxTypes = [[5,10],[2,5],[4,7],[3,9]], truckSize = 10
> - Only way to get to 91 would be to take the 50 from the 5 boxes of 10.
> - Then take 27 from the 3 boxes of 9 (77 in 8 boxes) 
> - Then take 2 boxes of 7 for 14 (91 in 10 boxes)
> - (5 * 10) + (3 * 9) + (2 * 7) => 50 + 27 + 14
>Output: 91

Hmm how about that they took from the biggest boxes in decending order again.

Conclusion: **Yes**, take the biggest box every time.

Okay so how do we get the biggest box?

Sort it? Seems good let's try that.

*But first*, check yourself with python weirdness:

In [1]:
box_types = [[5,10],[2,5],[4,7],[3,9]]
also_box_types = box_types.sort()
# What is it? Do you know?
print(box_types)
print(also_box_types)

[[2, 5], [3, 9], [4, 7], [5, 10]]
None


In [2]:
box_types = [[5,10],[2,5],[4,7],[3,9]]
box_types.sort(key=lambda x: x[1], reverse=True)
# How about now?
print(box_types)

[[5, 10], [3, 9], [4, 7], [2, 5]]


We're going to assume a couple things for our first pass.

1. Python sorted this using [TimSort](https://en.wikipedia.org/wiki/Timsort). For our purposes, you can think of that as an O(N<sub>log</sub>N) algorithm<sup>[1]</sup>. This will factor into our final runtime. 
2. We're allowed to modify the array in place. Not all problems or interviewers would allow this. *If they don't*, then make a copy instead, e.g.:
   ```python
   boxes = sorted(box_types, key=lambda x:x[1], reverse=True)
   ```

[1]: Note that it doesn't really matter what sort Python *actually* used as long as it's also an O(N<sub>log</sub>N) algorithm. But at time of writing it was TimSort.

Okay so, sort it and then take as many of the biggest boxes as we can until the truck is full.

In [3]:
def maximum_units(box_types, truck_size) -> int:
    # Sort by box size.
    box_types.sort(key=lambda x: x[1], reverse=True)

    units = 0
    for num_box, units_per_box in box_types:
        # We can't take more boxes than we can fit in our truck!
        boxes_to_take = boxes_to_take = min(num_box, truck_size) 
        units += (boxes_to_take * units_per_box)
        # Reduce remaining truck size.
        truck_size -= boxes_to_take

    return units

assert maximum_units([[5,10],[2,5],[4,7],[3,9]], 10) == 91

And you know what? It turns out this solution is *pretty good*!

At time of writing it beats 93.82% in Runtime and 96.33% in Memory.

### But I wanna go faster

But hold on. Let's stop for a second. What's the actual time complexity of what we have?

In [4]:
def maximum_units(box_types, truck_size) -> int:
    # Assuming len(box_types)==n
    box_types.sort(key=lambda x: x[1], reverse=True) # O(n logn)

    units = 0
    for num_box, units_per_box in box_types: # O( len(n) * 
        # We can't take more boxes than we can fit in our truck!
        boxes_to_take = boxes_to_take = min(num_box, truck_size) # O(1)
        units += (boxes_to_take * units_per_box)                 # O(1) 
        truck_size -= boxes_to_take                              # O(1))
    
    # Loop complexity -> O(n)
    # Total complexity -> O(n) + O(n logn) -> O(n logn)
    return units

assert maximum_units([[5,10],[2,5],[4,7],[3,9]], 10) == 91

We have an O(n<sub>log</sub>n) algorithm already! That's... pretty good.

If you were writing code for a job and stopped here, I would not blame you.

We're just gonna try to cut down on runtime for the hell of it.

With that all said, we gotta shave time off that \*checks notes\* hold on while I run this cell.

In [5]:
%%timeit

maximum_units([[5,10],[2,5],[4,7],[3,9]], 10)

1.17 μs ± 4.68 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


We gotta shave time off that \*squints\* microsecond long function call.

Because of a Computer Science principle called ~"Fucking Magic"~ the "[Time-Memory Trade-Off](https://en.wikipedia.org/wiki/Space%E2%80%93time_tradeoff)", we can probably leverage extra memory to decrease runtime. 

Do you care if a solution that is twice as fast uses twice as much memory when you're sitting on 64GB of RAM? Probably not.

Do you care if something takes 0.6μs instead of 1.2μs? ~Definitely not but-~

## Okay so how do we Go Faster

I've actually tricked you. You see, we've been ignoring the magic trick of this problem, and unfortunately this problem was written by someone who thinks it's funny to occasionally split the boxes, despite the fact that that makes no sense.

That means that these two are both syntactically correct ways to refer to the same batch boxes, even though the second one is insane:
> [[5,10],[2,5],[4,7],[3,9]]
> 
> [[1,10],[1,10],[1,10],[1,10],[1,10],[1,5],[1,5],[1,7],[1,7],[1,7],[1,7],[1,9],[1,9],[1,9]]

As the cell below demonstrates, this is quite a bit slower with our current algorithm.

In [6]:
%%timeit
maximum_units([[1,10],[1,10],[1,10],[1,10],[1,10],[1,5],[1,5],[1,7],[1,7],[1,7],[1,7],[1,9],[1,9],[1,9]], 10)

3.51 μs ± 16.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


That also means *that's* where our timesave actually is. Getting all valid inputs of boxes as close to the floor of ~1μs as possible, which is ~1/3 the running time of our code on insane inputs.

(Yeah, I hate it too.)

The good news is we already have a really solid foundation to work with! Plus, we know what we can do to speed it up. If we can leverage extra space to condense any input down to it's tightest representation, then we can use our PrettyGood<sup>TM</sup> algorithm and be on our merry way.

Or, to codify that:

In [7]:
def maximum_units(boxTypes, truckSize) -> int:
    def ensure_sane_input(box_types):
        # This is the part that we need to implement
        # Note that in our final code these won't be separate helper functions anymore.
        return box_types

    # Original algorithm, from above.
    def fit_to_truck(box_types, truck_size):
        box_types.sort(key=lambda x: x[1], reverse=True)
        units = 0
        for num_box, units_per_box in box_types:
            boxes_to_take = boxes_to_take = min(num_box, truck_size)
            units += boxes_to_take * units_per_box
            truck_size -= boxes_to_take
        return units
    
    return fit_to_truck(ensure_sane_input(boxTypes), truckSize)

assert maximum_units([[5,10],[2,5],[4,7],[3,9]], 10) == 91

There's a few options here. My original solution used a dictionary, because dictionaries are great, and can be used for everything. But in this specific case, we can use the dictionary construct to condense all the box entries back together based on the units they contain.

In [8]:
def maximum_units(box_types, truck_size):
    better_boxes = {}

    # +1 point here for recognizing I could have used a defaultdict.
    for num_box, units_per_box in box_types:
        try:
            better_boxes[units_per_box] += num_box
        except KeyError:
            better_boxes[units_per_box] = num_box

    queue = [k for k in better_boxes.keys()]
    queue.sort(reverse=True)

    units = 0
    for item in queue:
        boxes_to_take = min(better_boxes[item], truck_size)
        units += item * boxes_to_take
        truck_size -= boxes_to_take
    
    return units

assert maximum_units([[5,10],[2,5],[4,7],[3,9]], 10) == 91

In [9]:
%%timeit
maximum_units([[1,10],[1,10],[1,10],[1,10],[1,10],[1,5],[1,5],[1,7],[1,7],[1,7],[1,7],[1,9],[1,9],[1,9]], 10)

3.22 μs ± 17.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Well that's disappointing! It wen't down, but not by much.

Nah, just kidding! I tricked you again.

You see, as mentioned earlier, Leetcode inputs are frequently written by insane people.

That was NOT an insane input, it's a mere shadow.

This code block below is an insane input:

In [10]:
# THIS is an insane input.
def two_to_the(n):
    return 2 ** n
    
why = []
why.extend([1, 127] for i in range(two_to_the(8)))
why.extend([1, 255] for i in range(two_to_the(9)))
why.extend([1, 512] for i in range(two_to_the(13)))
why.extend([1, 127] for i in range(two_to_the(15)))
why.extend([1, 1024] for i in range(two_to_the(12)))
# This is somehow just shy of 46% of what we COULD get.
truck_size = 2 ** 13

print(len(why))

# Verify later functions all have the expected output.
expected = maximum_units(why, truck_size)
print(expected)

45824
6291456


So let's try our first couple implementations against this new input.

In [11]:
def initial_solution(box_types, truck_size) -> int:
    box_types.sort(key=lambda x: x[1], reverse=True)
    
    units = 0
    for num_box, units_per_box in box_types:
        boxes_to_take = boxes_to_take = min(num_box, truck_size)
        units += boxes_to_take * units_per_box
        truck_size -= boxes_to_take

    return units

def dict_joke(box_types, truck_size):
    better_boxes = {}
    for num_box, units_per_box in box_types:
        try:
            better_boxes[units_per_box] += num_box
        except KeyError:
            better_boxes[units_per_box] = num_box
    queue = [k for k in better_boxes.keys()]
    queue.sort(reverse=True)

    units = 0
    for item in queue:
        boxes_to_take = min(better_boxes[item], truck_size)
        units += item * boxes_to_take
        truck_size -= boxes_to_take
    
    return units

assert initial_solution(why, truck_size) == expected
assert dict_joke(why, truck_size) == expected

In [12]:
%%timeit
initial_solution(why, truck_size)

9.72 ms ± 85 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
%%timeit
dict_joke(why, truck_size)

3.51 ms ± 19.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


So with awful AWFUL input, we can see that our dict_cache_solution is more than 2.5x as fast ~on my machine~!

We may want to consider other strategies as well. Here's two:
- What about a heap? We could fill our queue while we condense input by sorting by the units.
- What if we made our queue while we built the cache by insorting the values to the right places? ~Wait that's just the heap again.~


In [14]:
from heapq import heappop, heappush

def heaping_dict(box_types, truck_size):
    better_boxes = {}
    queue = []
    remaining_space = int(truck_size)

    for num_box, units_per_box in box_types:
        try:
            better_boxes[units_per_box] += num_box
        except KeyError:
            better_boxes[units_per_box] = num_box
            # Mildly cursed inverse to make it a max heap
            heappush(queue, -units_per_box)

    units = 0
    while remaining_space:
        # uncurse value
        item = -heappop(queue)
        boxes_to_take = min(better_boxes[item], truck_size)
        units += item * boxes_to_take
        truck_size -= boxes_to_take

        if not queue:
            break
    
    return units


from bisect import insort

def bisect_dict(box_types, truck_size):
    better_boxes = {}
    queue = []

    for num_box, units_per_box in box_types:
        try:
            better_boxes[units_per_box] += num_box
        except KeyError:
            better_boxes[units_per_box] = num_box
            insort(queue, units_per_box) 

    units = 0
    for item in iter(queue[::-1]):
        boxes_to_take = min(better_boxes[item], truck_size)
        units += item * boxes_to_take
        truck_size -= boxes_to_take
    
    return units

assert bisect_dict(why, truck_size) == expected
assert heaping_dict(why, truck_size) == expected

In [15]:
%%timeit
heaping_dict(why, truck_size)

3.55 ms ± 19.3 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%%timeit
bisect_dict(why, truck_size)

3.58 ms ± 59.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


At least on my machine, it seems like all these solutions are roughly equal. Notably, the `dict_joke` version has the lowest std. dev. between the variations on my machine, but they're all roughly 3.50ms.

Which kind of makes sense, right? With a basic pattern of "Iterate, sort, iterate", we're not able to do do much better even if we do change around our structures. Maybe we could implement some non-comparitive sort like [Radix Sort](https://en.wikipedia.org/wiki/Radix_sort) but doing that in Python sounds like the kind of thing that would make me start drinking on the job.

This isn't every way you could solve this problem, but I do wanna to hammer in that bit about complexity analysis again. Notice how these are ALL O(n<sub>log</sub>n)? I think there's two takeaways from this:
- Not all algorithms of the same complexity class have similar real world runtimes.
- Implementation details do matter but a "Good Enough" implementation is probably... good enough.

And that's everything I care to know about this Leetcode problem. Sorry for tricking you earlier. <sub>Twice.</sub>

Happy coding!